In [1]:
import $file.common
import spark._
import common._
import org.apache.spark.sql.functions._
import org.apache.spark.rdd._
import org.apache.spark.sql.types.{IntegerType, StringType, StructType}
import org.apache.spark.sql.functions.{col, to_date}
import spark.implicits._
import spark.sqlContext.implicits._

Loading spark-stubs
Getting spark JARs


log4j:WARN No appenders could be found for logger (org.eclipse.jetty.util.log).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
21/07/05 16:25:23 INFO SparkContext: Running Spark version 2.4.5
21/07/05 16:25:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/07/05 16:25:24 INFO SparkContext: Submitted application: Queries Optimization
21/07/05 16:25:24 INFO SecurityManager: Changing view acls to: jovyan
21/07/05 16:25:24 INFO SecurityManager: Changing modify acls to: jovyan
21/07/05 16:25:24 INFO SecurityManager: Changing view acls groups to: 
21/07/05 16:25:24 INFO SecurityManager: Changing modify acls groups to: 
21/07/05 16:25:24 INFO SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(jovyan); groups with view permissions: Set(); users  with modify permissions: Set(jovyan); groups with modify permissions: Set()
21/07/05 16:25:25 INFO Utils: Successfully started service 'sparkDriver' on port 3

21/07/05 16:25:26 INFO SparkContext: Added JAR file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/org/scala-lang/modules/scala-collection-compat_2.12/2.0.0/scala-collection-compat_2.12-2.0.0-sources.jar at spark://98858bd2f6bb:35335/jars/scala-collection-compat_2.12-2.0.0-sources.jar with timestamp 1625502326511
21/07/05 16:25:26 INFO SparkContext: Added JAR file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/org/scala-lang/modules/scala-collection-compat_2.12/2.0.0/scala-collection-compat_2.12-2.0.0.jar at spark://98858bd2f6bb:35335/jars/scala-collection-compat_2.12-2.0.0.jar with timestamp 1625502326511
21/07/05 16:25:26 INFO SparkContext: Added JAR file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/org/scala-lang/modules/scala-xml_2.12/1.2.0/scala-xml_2.12-1.2.0-sources.jar at spark://98858bd2f6bb:35335/jars/scala-xml_2.12-1.2.0-sources.jar with timestamp 1625502326512
21/07/05 16:25:26 INFO SparkContext: Added JAR file:/home/jovyan/.cache

21/07/05 16:25:26 INFO SparkContext: Added JAR file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/com/fasterxml/jackson/module/jackson-module-scala_2.12/2.9.9/jackson-module-scala_2.12-2.9.9.jar at spark://98858bd2f6bb:35335/jars/jackson-module-scala_2.12-2.9.9.jar with timestamp 1625502326533
21/07/05 16:25:26 INFO SparkContext: Added JAR file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/com/fasterxml/jackson/module/jackson-module-paranamer/2.9.9/jackson-module-paranamer-2.9.9.jar at spark://98858bd2f6bb:35335/jars/jackson-module-paranamer-2.9.9.jar with timestamp 1625502326534
21/07/05 16:25:26 INFO SparkContext: Added JAR file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/com/squareup/okhttp3/logging-interceptor/3.11.0/logging-interceptor-3.11.0.jar at spark://98858bd2f6bb:35335/jars/logging-interceptor-3.11.0.jar with timestamp 1625502326534
21/07/05 16:25:26 INFO SparkContext: Added JAR file:/home/jovyan/.cache/coursier/v1/https/repo1.

Spark UI

import $file.$     

import spark._

import common._

import org.apache.spark.sql.functions._

import org.apache.spark.rdd._

import org.apache.spark.sql.types.{IntegerType, StringType, StructType}

import org.apache.spark.sql.functions.{col, to_date}

import spark.implicits._

import spark.sqlContext.implicits._

In [2]:
import plotly._
import plotly.element._
import plotly.layout._
import plotly.Almond._

import plotly._

import plotly.element._

import plotly.layout._

import plotly.Almond._

# Query 2: Media de infecciones por Superficie

Esta query utiliza dos conjuntos de datos, por una parte los datos de casos reportados diariemente por país y por otro lado una tabla de datos demográficos por país en la que encontramos la superficie de cada país.

El objetivo de la query es:
    - Agrupar los datos por paises
    - Obtener la tasa de infectados diarios por superficie (en Km^{2})
        - Para ello dividimos los casos diarios entre la superficie
    - Calcular la media de infecciones por superficie
    
De esta forma obtendremos la media de contagios por Km^{2}

# 1. Utilizando RDDs

In [3]:
def infections(lines : RDD[String]) : RDD[Infection] =
    lines.map(line => {
      val arr = line.split(",")
      Infection(
        day = arr(1).toInt,
        month = arr(2).toInt,
        year = arr(3).toInt,
        nCases = arr(4).toInt,
        nDeaths = arr(5).toInt,
        country = arr(6),
        continent = arr(10)
      )
    })

defined function infections

In [4]:
def infectionRDD = infections(spark.sparkContext.textFile("../datasets/data.csv",8))

defined function infectionRDD

In [5]:
org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this)
case class Population(
    country : String, 
    population : Int, 
    density : Int, 
    land_area: Int, 
    ) 
extends Serializable

defined class Population

In [6]:
def populationData = spark.sparkContext.textFile("../datasets/population_by_country_2020.csv",8)

defined function populationData

In [7]:
def population(lines : RDD[String]) : RDD[Population] =
    lines.mapPartitionsWithIndex(
                   (index, it) => if (index == 0) it.drop(1) else it,
                    preservesPartitioning = true
                 )
    .map(line => {
      val arr = line.split(",")
      Population(
        country = arr(0),
        population = arr(1).toInt,
        density = arr(4).toInt,
        land_area = arr(5).toInt,
      )
    })

defined function population

In [8]:
def populationRDD = population(populationData)

defined function populationRDD

### 1.1 RDD Sin optimizar

Aquí vemos una de las grandes desventajas de los RDDs, que al ser utilizados como colecciones y creados "a mano" no se aplican todas las optimizaciones que se pueden aplicar como en DF o DS gracias a Catalyst

Por ejemplo:

Un join computacionalmente pesado desde el principio ya que cruza todos los datos sin 
quedarnos con los que nos interesen

Spark no me deja hacer un Join de RDD que no sean pair RDD así que tenemos que construirlo

In [8]:
// populationRDD.join(infectionRDD)

Construyo Pair RDDs conservando todos los datos

In [9]:
def populationByCountry = populationRDD.map(
    x => (x.country,x))

def infectionByCountry = infectionRDD.map(
    x => (x.country,x))

defined function populationByCountry
defined function infectionByCountry

Hago el Join y agrupo por paises

In [10]:
def joinedRDD = infectionByCountry.join(populationByCountry).groupByKey()

defined function joinedRDD

Finalmente calculo la media

In [11]:
def meanRDD = joinedRDD.mapValues(
    x => x.map( 
        line => line._1.nCases.toDouble / line._2.land_area.toDouble
    )).mapValues(
    x => x.sum / x.size
)

defined function meanRDD

Lo hago todo en una única operación para calcular el tiempo de ejecución

In [12]:
def query_RDD_Not_Optimized =
    infectionByCountry.join(populationByCountry)
    .groupByKey()
    .mapValues(
        x => x.map( 
            line => line._1.nCases.toDouble / line._2.land_area.toDouble))
    .mapValues(
        x => x.sum / x.size
    )

defined function query_RDD_Not_Optimized

### 1.2 Para optimizar un poco este RDD:

Despejo solo los datos que me interesan para trabajar con Pair RDDs y optimizar la consulta

In [13]:
def countriesAndLandArea = populationRDD.map(
    x => (x.country,x.land_area))

defined function countriesAndLandArea

In [14]:
def countriesAndCases = infectionRDD.map(
    x => (x.country,x.nCases))

defined function countriesAndCases

Ejecuto un join y trabajo para calcular primero la media de infecciones por Km2 diaria, 
para luego calcular la media total

In [15]:
def average = countriesAndCases.join(countriesAndLandArea).groupByKey()

defined function average

In [16]:
def avg = average.mapValues(
    x => x.map(
        line => (line._1.toDouble / line._2.toDouble)
    )).mapValues(
    x => x.sum/x.size
)

defined function avg

Lo hago todo en una única operación para calcular el tiempo de ejecución

In [17]:
def query_RDD =
    countriesAndCases.join(countriesAndLandArea)
    .groupByKey()
    .mapValues(
        x => x.map(
        line => (line._1.toDouble / line._2.toDouble)))
    .mapValues(
        x => x.sum / x.size
)

defined function query_RDD

### 1.3 utilizando reduce

In [18]:
def query_RDD_Optimized = 
        infectionRDD.map(x => (x.country,(1,x.nCases)))
        .reduceByKey((x,y) => (x._1 + y._1, x._2 + y._2))
        .map{ case(country, (days,cases)) => (country, cases/days)}
        .join(countriesAndLandArea)
        .map(x => (x._1, x._2._1.toDouble / x._2._2.toDouble))

defined function query_RDD_Optimized

# 2. Consulta con DataSet

In [19]:
spark.conf.set("spark.sql.shuffle.partitions", 8)

In [34]:
val infectionDS = spark.read
    .option("header", "true")
    .option("charset", "UTF8")
    .option("delimiter",",")
    .option("inferSchema", "true")
    .csv("../datasets/covidworldwide.csv")
    .withColumnRenamed("countriesAndTerritories","Country")
    .as[(String,String,String,String,Double,Double,String,String,String,String,String,String)]

csv at cmd33.sc:6

1 / 1

csv at cmd33.sc:6

2 / 2

infectionDS: org.apache.spark.sql.Dataset[(String, String, String, String, Double, Double, String, String, String, String, String, String)] = [dateRep: string, day: int ... 10 more fields]

In [35]:
val populationDS = spark.read
    .option("header", "true")
    .option("charset", "UTF8")
    .option("delimiter",",")
    .option("inferSchema", "true")
    .csv("../datasets/population_by_country_2020.csv")
    .withColumnRenamed("Country (or dependency)","Country")
    .withColumnRenamed("Population (2020)","Population")
    .withColumnRenamed("Land Area (Km\u00b2)","Area")
    .as[(String,Float,String,Float,Float,Float,Double,String,String,String,String)]

csv at cmd34.sc:6

1 / 1

csv at cmd34.sc:6

1 / 1

populationDS: org.apache.spark.sql.Dataset[(String, Float, String, Float, Float, Float, Double, String, String, String, String)] = [Country: string, Population: int ... 9 more fields]

In [22]:
def query_DS = 
    infectionDS.join(populationDS, "Country")
            .select($"Country",
                    $"dateRep" as "date",
                    $"cases",
                    $"Area",
                    $"cases" / $"Area" as "infection Per Km\u00b2")
            .groupBy("Country")
            .avg("infection Per Km\u00b2")
            .orderBy(desc("avg(infection Per Km²)"))
            .as[(String,Double)]

defined function query_DS

# 3. Consulta con DataFrame

In [36]:
val dfCovid = spark.read
    .option("header", "true")
    .option("charset", "UTF8")
    .option("delimiter",",")
    .option("inferSchema", "true")
    .csv("../datasets/covidworldwide.csv")

csv at cmd35.sc:6

1 / 1

csv at cmd35.sc:6

2 / 2

dfCovid: org.apache.spark.sql.package.DataFrame = [dateRep: string, day: int ... 10 more fields]

In [37]:
val dfPopulation = spark.read
    .option("header", "true")
    .option("charset", "UTF8")
    .option("delimiter",",")
    .option("inferSchema", "true")
    .csv("../datasets/population_by_country_2020.csv")
    .withColumnRenamed("Country (or dependency)","Country")
    .withColumnRenamed("Population (2020)","Population")

csv at cmd36.sc:6

1 / 1

csv at cmd36.sc:6

1 / 1

dfPopulation: org.apache.spark.sql.package.DataFrame = [Country: string, Population: int ... 9 more fields]

Modifico los datos de entrada para que el formato fecha se adecue al TimeStamp de Spark

In [25]:
def dfCovidClean = dfCovid
    .select($"*",$"dateRep",translate($"dateRep","/","-").as("date"))
    .drop("dateRep")

defined function dfCovidClean

In [26]:
def dfCovidDate = dfCovidClean
    .select($"*",col("date"),to_date(col("date"),"dd-MM-yyyy").as("to_date"))

defined function dfCovidDate

Hago una consulta de prueba para obtener la media solo de los casos en España

In [27]:
def spainCovid = dfCovid
    .select("dateRep","cases")
    .where("countriesAndTerritories == 'Spain'").toDF

defined function spainCovid

### Finalmente ejecuto la consulta de nuestro caso de uso, infecciones por Km2

In [40]:
def query_DF = 
    dfCovid.join(dfPopulation, $"country" === $"countriesAndTerritories")
            .select($"country",
                    $"dateRep" as "date",
                    $"cases",
                    $"Land Area (Km\u00b2)",
                    $"cases" / $"Land Area (Km\u00b2)" as "infection Per Km\u00b2")
            .groupBy("country")
            .avg("infection Per Km\u00b2")
            .orderBy(desc("avg(infection Per Km²)"))

defined function query_DF

# 4. Visualización de rendimiento

In [39]:
val (x, y) = Seq(
    "Not Optimized RDD" -> runWithOutput(query_RDD_Not_Optimized.collect()),
    "RDD" -> runWithOutput(query_RDD.collect()),
    "RDD Optimized" -> runWithOutput(query_RDD_Optimized.collect()),
    "DataSet" -> runWithOutput(query_DS.collect),
    "DataFrame" -> runWithOutput(query_DF.collect)
).unzip

Bar(x, y).plot()

map at cmd8.sc:4

8 / 8

map at cmd8.sc:1

8 / 8

collect at cmd38.sc:2

...

8 / 8

collect at cmd38.sc:6

8 / 8

Took 1061


x: Seq[String] = List(
  "Not Optimized RDD",
  "RDD",
  "RDD Optimized",
  "DataSet",
  "DataFrame"
)
y: Seq[Int] = List(632, 471, 372, 1123, 1061)
res38_1: String = "plot-5c019c1a-03ca-4a72-aaf7-1e14d9284b8a"

# 5. Comparativas de rendimiento

In [29]:
spark.time(query_RDD_Not_Optimized.collect())

map at cmd8.sc:1

8 / 8

map at cmd8.sc:4

8 / 8

collect at cmd28.sc:1

8 / 8

Time taken: 5109 ms


res28: Array[(String, Double)] = Array(
  ("Australia", 1.0425077311296284E-5),
  ("Israel", 0.047656836789593385),
  ("Eswatini", 0.001430866807610994),
  ("Peru", 0.0027287594193262437),
  ("Netherlands", 0.05191882731740392),
  ("Benin", 1.0037851470983781E-4),
  ("Morocco", 0.0031977929643737385),
  ("Montenegro", 0.0113235294117647),
  ("Belarus", 0.002309885320631594),
  ("Cambodia", 5.9641169485251785E-6),
  ("Moldova", 0.013754946727549473),
  ("Mongolia", 2.1424773766604925E-6),
  ("Gibraltar", 0.39814814814814803),
  ("Kazakhstan", 2.5090599292177247E-4),
  ("Vietnam", 1.3021486291273772E-5),
  ("Dominica", 4.3445692883895145E-4),
  ("Taiwan", 5.9708718279743496E-5),
  ("Tajikistan", 3.981087328209058E-4),
  ("Panama", 0.009272406702794105),
  ("Comoros", 0.0014865770143947046),
  ("Vanuatu", 2.4127780726728754E-6),
  ("Chad", 5.206127347169282E-6),
  ("Ethiopia", 4.2307608695652194E-4),
  ("Niger", 6.626708806039057E-6),
  ("Iraq", 0.0038019136269036207),
  ("Botswana", 8.54

In [30]:
spark.time(query_RDD.collect())

map at cmd13.sc:1

8 / 8

map at cmd12.sc:1

8 / 8

collect at cmd29.sc:1

8 / 8

Time taken: 658 ms


res29: Array[(String, Double)] = Array(
  ("Australia", 1.0425077311296284E-5),
  ("Israel", 0.047656836789593385),
  ("Eswatini", 0.001430866807610994),
  ("Peru", 0.0027287594193262437),
  ("Netherlands", 0.05191882731740392),
  ("Benin", 1.0037851470983781E-4),
  ("Morocco", 0.0031977929643737385),
  ("Montenegro", 0.0113235294117647),
  ("Belarus", 0.002309885320631594),
  ("Cambodia", 5.9641169485251785E-6),
  ("Moldova", 0.013754946727549473),
  ("Mongolia", 2.1424773766604925E-6),
  ("Gibraltar", 0.39814814814814803),
  ("Kazakhstan", 2.5090599292177247E-4),
  ("Vietnam", 1.3021486291273772E-5),
  ("Dominica", 4.3445692883895145E-4),
  ("Taiwan", 5.9708718279743496E-5),
  ("Tajikistan", 3.981087328209058E-4),
  ("Panama", 0.009272406702794105),
  ("Comoros", 0.0014865770143947046),
  ("Vanuatu", 2.4127780726728754E-6),
  ("Chad", 5.206127347169282E-6),
  ("Ethiopia", 4.2307608695652194E-4),
  ("Niger", 6.626708806039057E-6),
  ("Iraq", 0.0038019136269036207),
  ("Botswana", 8.54

In [31]:
spark.time(query_RDD_Optimized.collect())

map at cmd17.sc:2

8 / 8

map at cmd12.sc:1

8 / 8

map at cmd17.sc:4

8 / 8

collect at cmd30.sc:1

8 / 8

Time taken: 619 ms


res30: Array[(String, Double)] = Array(
  ("Australia", 1.0413548025981803E-5),
  ("Israel", 0.04764325323475046),
  ("Eswatini", 0.0013953488372093023),
  ("Peru", 0.002728125),
  ("Netherlands", 0.051897983392645314),
  ("Benin", 9.755232351897836E-5),
  ("Morocco", 0.0031974008514452164),
  ("Montenegro", 0.011301115241635688),
  ("Belarus", 0.002306441279384949),
  ("Cambodia", 5.665080444142307E-6),
  ("Moldova", 0.013729071537290716),
  ("Mongolia", 1.9310486881742578E-6),
  ("Gibraltar", 0.3),
  ("Kazakhstan", 2.5076860391895397E-4),
  ("Vietnam", 1.290031283258619E-5),
  ("Dominica", 0.0),
  ("Taiwan", 5.648121999435188E-5),
  ("Tajikistan", 3.9296941983423835E-4),
  ("Panama", 0.009268227064837235),
  ("Comoros", 0.0010746910263299302),
  ("Vanuatu", 0.0),
  ("Chad", 4.7649301143583225E-6),
  ("Ethiopia", 4.23E-4),
  ("Niger", 6.315623273071762E-6),
  ("Iraq", 0.003801344630687051),
  ("Botswana", 8.469641628288603E-5),
  ("Lithuania", 0.004435651147206178),
  ("Laos", 0.0),
 

In [38]:
spark.time(query_DS.collect())

csv at cmd19.sc:6

1 / 1

csv at cmd19.sc:6

2 / 2

csv at cmd20.sc:6

...

8 / 8

collect at cmd37.sc:1

8 / 8

Time taken: 1203 ms


res37: Array[(String, Double)] = Array(
  ("Monaco", 1.9763313609467457),
  ("Gibraltar", 0.39814814814814803),
  ("Bahrain", 0.3360843010104055),
  ("Singapore", 0.23804081632653054),
  ("Maldives", 0.15914285714285714),
  ("Malta", 0.12258171378091859),
  ("Aruba", 0.10505617977528105),
  ("Belgium", 0.05736940932251372),
  ("Andorra", 0.05656799259944492),
  ("Netherlands", 0.05191882731740392),
  ("Guam", 0.048558152248189136),
  ("Israel", 0.047656836789593385),
  ("Luxembourg", 0.04645811992750769),
  ("Lebanon", 0.04139474174902102),
  ("Qatar", 0.03509058863945274),
  ("Liechtenstein", 0.03391577060931893),
  ("Bermuda", 0.03192592592592586),
  ("Cura\u00e7ao", 0.030684820158504333),
  ("Switzerland", 0.026920668662241683),
  ("Kuwait", 0.023646325567555148),
  ("Italy", 0.01790898406006858),
  ("Slovenia", 0.016838818411961356),
  ("Armenia", 0.015314984029080356),
  ("Moldova", 0.013754946727549473),
  ("Bangladesh", 0.013703283119277588),
  ("Portugal", 0.013267124014649443)

In [41]:
spark.time(query_DF.collect())

run at ThreadPoolExecutor.java:1149

1 / 1

collect at cmd40.sc:1

2 / 2

collect at cmd40.sc:1

...

8 / 8

collect at cmd40.sc:1

8 / 8

Time taken: 531 ms


res40: Array[org.apache.spark.sql.Row] = Array(
  [Monaco,1.9763313609467457],
  [Gibraltar,0.39814814814814803],
  [Bahrain,0.3360843010104055],
  [Singapore,0.23804081632653054],
  [Maldives,0.15914285714285714],
  [Malta,0.12258171378091859],
  [Aruba,0.10505617977528105],
  [Belgium,0.05736940932251372],
  [Andorra,0.05656799259944492],
  [Netherlands,0.05191882731740392],
  [Guam,0.048558152248189136],
  [Israel,0.047656836789593385],
  [Luxembourg,0.04645811992750769],
  [Lebanon,0.04139474174902102],
  [Qatar,0.03509058863945274],
  [Liechtenstein,0.03391577060931893],
  [Bermuda,0.03192592592592586],
  [Curaçao,0.030684820158504333],
  [Switzerland,0.026920668662241683],
  [Kuwait,0.023646325567555148],
  [Italy,0.01790898406006858],
  [Slovenia,0.016838818411961356],
  [Armenia,0.015314984029080356],
  [Moldova,0.013754946727549473],
  [Bangladesh,0.013703283119277588],
  [Portugal,0.013267124014649443],
  [Poland,0.013058352071742075],
  [France,0.012402372199215262],
  [Mont

In [ ]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(query_RDD_Not_Optimized.collect)

In [ ]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(query_RDD.collect)

In [ ]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(query_DS.collect)

In [ ]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(
    query_DF.collect
    )

# 6. Visualización de datos con plotly

In [ ]:
val (x,y) = query_DF
    .collect
    .map(r=>(r(0).toString, r(1).toString.toFloat))
    .toList
    .unzip

Bar(x, y).plot()